<a href="https://colab.research.google.com/github/Hubert26/machine-learning/blob/main/project_hubert_szewczyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPIS PROJEKTU

Lending Club to firma pożyczkowa typu peer-to-peer, która łączy pożyczkobiorców z
inwestorami za pośrednictwem platformy internetowej. Obsługuje osoby, które potrzebują
pożyczek osobistych w wysokości od 1000 do 40 000 USD. Pożyczkobiorcy otrzymują pełną
kwotę udzielonej pożyczki pomniejszoną o opłatę początkową, która jest uiszczana firmie.
Inwestorzy kupują weksle zabezpieczone osobistymi pożyczkami i płacą Lending Club
opłatę za usługę. Firma Lending Club udostępnia dane o wszystkich pożyczkach
udzielonych za pośrednictwem swojej platformy w określonych okresach.
Na potrzeby tego projektu zostały użyte dane dotyczące pożyczek udzielonych za
pośrednictwem Lending Club na przestrzeni lat 2007 -2011. Każda pożyczka jest opatrzona
informacją o tym, czy ostatecznie została spłacona (Fully Paid lub Charged off w kolumnie
loan_status).

W tym projekcie chcę zbudować model klasyfikacyjny, który na podstawie zebranych danych będzie przewidywał, czy potencjalny pozyczkobiorca spłaci swój dług.

#KOD

##Import bibliotek i ustawienia

In [105]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

##Definicje funkcji

In [106]:
def display_info(df, selected_columns, df_info):
    for col_name in selected_columns:
        print("Column Name:", col_name)

        # Description
        if col_name in df_info.index:
            print("Description:", df_info.loc[col_name]['Description'])
        else:
            print("Description not available.")

        # Statistics
        print("\nStatistics:")
        print(df[col_name].describe())

        # Type
        print("\nType:")
        print(df[col_name].dtype)

        # Mode
        print("\nMode:")
        print(df[col_name].mode())

        # NaN count
        nan_count = df[col_name].isna().sum()
        print("\nNaN count:", nan_count)

        print("\n" + "=" * 40)  # Separator between columns

##1. Obróbka danych

###Pobranie danych z plików

In [107]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/Loan_data.csv', low_memory=False)

In [108]:
df = df_raw.copy()

In [109]:
df_info_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/LCDataDictionary.csv')

In [110]:
df_info = df_info_raw.iloc[:, 0:2]
df_info.set_index(df_info.columns.tolist()[0], inplace=True)

###Eksploracja danych

In [111]:
df.shape

(42536, 151)

In [112]:
df.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.0,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42536 entries, 0 to 42535
Columns: 151 entries, id to settlement_term
dtypes: float64(120), object(31)
memory usage: 49.0+ MB


In [114]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,42535.000000,42506.000000,42535.000000,42535.000000,42506.000000,15609.000000,3651.000000,42506.000000,42506.000000,4.253500e+04,42506.000000,42535.0,42535.0,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42390.0,0.0,42535.0,0.0,0.0,0.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42390.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41170.000000,42430.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,160.000000,160.000000
mean,NaN,11089.722581,10821.585753,10139.938785,322.623063,6.913656e+04,13.373043,0.152449,713.052545,717.052545,1.081424,35.017618,59.175568,9.343951,0.058156,1.429786e+04,22.124406,0.0,0.0,12020.189644,11312.729377,9675.675936,2239.981444,1.516890,103.015467,14.387843,2613.245652,689.922511,676.952039,0.0,NaN,1.0,NaN,NaN,NaN,0.000094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.143039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045227,0.000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4272.137875,49.905875,1.200000
std,NaN,7410.938391,7146.914675,7131.598014,208.927216,6.409635e+04,6.726315,0.512406,36.188439,36.188439,1.527455,22.418427,47.145396,4.496274,0.245713,2.201844e+04,11.592811,0.0,0.0,9094.685888,9038.506549,7105.750304,2585.057393,7.830064,735.799086,162.177837,4385.066535,80.818099,119.647752,0.0,NaN,0.0,NaN,NaN,NaN,0.009700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.359579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [115]:
#Sprawdzenie czy wszystkie wartości są unikalne
len(df['id'].unique())==df.shape[0]

True

In [116]:
column_names = df.columns.tolist()

In [117]:
mask = df.isna()

In [118]:
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
df_info = df_info.merge(naan_column.T, left_index=True, right_index=True, how='left')

In [119]:
mode_column = pd.DataFrame(columns=['mode'], index=column_names)
for col_name in column_names:
    if not df[col_name].empty:
        mode_value = df[col_name].mode()
        if not mode_value.empty:
            mode_column.loc[col_name, 'mode'] = mode_value.iloc[0]
        else:
            mode_column.loc[col_name, 'mode'] = None
    else:
        mode_column.loc[col_name, 'mode'] = None
df_info = df_info.merge(mode_column, left_index=True, right_index=True, how='left')

In [120]:
unique_len_column = pd.DataFrame(columns=['unique_len'], index=column_names)
for col_name in column_names:
    if not df[col_name].empty:
        unique_len_column.loc[col_name, 'unique_len'] = len([value for value in df[col_name].unique() if not pd.isna(value)])
    else:
        unique_len_column.loc[col_name, 'unique_len'] = 0
df_info = df_info.merge(unique_len_column, left_index=True, right_index=True, how='left')

In [121]:
df_type_column = pd.DataFrame(columns=['raw_column_type'], index=column_names)
for col_name in column_names:
  df_type_column.loc[col_name, 'raw_column_type'] = df[col_name].dtype

df_info = df_info.merge(df_type_column, left_index=True, right_index=True, how='left')

In [122]:
df_info = df_info.merge(pd.DataFrame(df.describe()).T, left_index=True, right_index=True, how='left')

In [123]:
df_info

,Description,NaN,mode,unique_len,raw_column_type,count,mean,std,min,25%,50%,75%,max
acc_now_delinq,The number of accounts on which the borrower i...,30.0,0.0,2,float64,42506.0,0.000094,0.009700,0.0,0.0,0.0,0.0,1.0
acc_open_past_24mths,Number of trades opened in past 24 months.,42536.0,None,0,float64,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
addr_state,The state provided by the borrower in the loan...,1.0,CA,50,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
all_util,Balance to credit limit on all trades,42536.0,None,0,float64,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,The self-reported annual income provided by th...,5.0,60000.0,5597,float64,42531.0,69136.556420,64096.349719,1896.0,40000.0,59000.0,82500.0,6000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
verification_status,"Indicates if income was verified by LC, not ve...",1.0,Not Verified,3,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
verified_status_joint,Indicates if the co-borrowers' joint income wa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zip_code,The first 3 numbers of the zip code provided b...,1.0,100xx,837,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Wyświetlenie Loan Status

Charged Off to status długu uznawanego za nieściągalny po 180 dniach braku płatności, ale nie oznacza umorzenia długu. Odpisanie długu wpływa negatywnie na historię kredytową i zdolność kredytową pożyczkobiorcy. Mimo spłaty długu, status Charged Off pozostaje w raporcie kredytowym przez siedem lat, choć może zostać zmieniony na "odpis opłacony" lub "odpis rozliczony". W praktyce, wiele osób spłaciło dług, ale nadal ma status Charged Off.

In [124]:
df_info.loc["settlement_status"]['Description']

'The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT'

In [125]:
df.settlement_status.unique()

array([nan, 'COMPLETE', 'BROKEN', 'ACTIVE'], dtype=object)

In [126]:
df_info.loc["loan_status"]['Description']

'Current status of the loan'

In [127]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [128]:
df["loan_status"] = df["loan_status"].replace(
    {"Does not meet the credit policy. Status:Fully Paid": "Fully Paid",
     "Does not meet the credit policy. Status:Charged Off": "Charged Off"})

In [129]:
df[(df.loan_status == 'Charged Off') & (df.settlement_status == "COMPLETE")].shape[0]

142

Wynika z tego, że są 142 wyniki z loan_status == Charged Off wraz z settlement_status == COMPLETE.

W dalszej części zmieniamy wynik Charged Off na Fully Paid dla takich wpisów.

In [130]:
df.loc[df['settlement_status'] == "COMPLETE", 'loan_status'] = "Fully Paid"

In [131]:
df['loan_status'].value_counts()

Fully Paid     36246
Charged Off     6289
Name: loan_status, dtype: int64

###Usunięcie kolumn które zawierają informacje z przyszłości, niedostępne w momencie udzielania pożyczki

In [132]:
future_data = df[["mths_since_last_delinq", "last_pymnt_d",
                      "collection_recovery_fee", "recoveries", "issue_d",
                      "total_rec_late_fee", "total_rec_prncp", "funded_amnt",
                      "funded_amnt_inv", "last_fico_range_high",
                      "last_fico_range_low", "debt_settlement_flag"]]
selected_columns = future_data.columns
df.drop(columns=selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn ze zbędnymi informacjami

In [133]:
selected_columns = df_info[df_info['NaN'] / df_raw.shape[0] > 0.9].index.tolist()
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn z tylko jedną unikalną wartością

In [134]:
selected_columns = df_info[df_info['unique_len'] < 2].index.tolist()
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn typu 'object' o bardzo dużej liczbie wartości z niewielką licznością pojedynczej wartości

In [135]:
df_type_column['raw_column_type'].value_counts()

float64    120
object      31
Name: raw_column_type, dtype: int64

In [136]:
for col_name in df.columns.tolist():
  if df[col_name].dtype == 'object':
    value_counts = df[col_name].value_counts()
    if len(value_counts) > 1 and value_counts.iloc[1] < 10:
      display(value_counts)

1077501    1
538025     1
540325     1
540300     1
540270     1
          ..
770036     1
769217     1
770000     1
765994     1
70686      1
Name: id, Length: 42536, dtype: int64

https://lendingclub.com/browse/loanDetail.action?loan_id=1077501    1
https://lendingclub.com/browse/loanDetail.action?loan_id=538025     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540325     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540300     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540270     1
                                                                   ..
https://lendingclub.com/browse/loanDetail.action?loan_id=770036     1
https://lendingclub.com/browse/loanDetail.action?loan_id=769217     1
https://lendingclub.com/browse/loanDetail.action?loan_id=770000     1
https://lendingclub.com/browse/loanDetail.action?loan_id=765994     1
https://lendingclub.com/browse/loanDetail.action?loan_id=70686      1
Name: url, Length: 42535, dtype: int64

In [137]:
selected_columns = {'url','id'}
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Przekształcenie wartości w kolumnach - usunięcie procentów, dodatkowych znaków, itd

In [138]:
df["revol_util"] = df["revol_util"].str[0:-1]
df["int_rate"] = df["int_rate"].str[0:-1]
df["revol_util"].astype(np.float64)
df["int_rate"].astype(np.float64)

df["zip_code"] = df["zip_code"].str[0:-2]

In [139]:
print("df after cleaning:", df.shape)

df after cleaning: (42536, 51)


In [140]:
selected_columns = df.columns.tolist()
for column_name in selected_columns:
    if column_name in df_info.index:
        description = df_info.loc[column_name, 'Description']
        print(f"{column_name} - {description}")
    else:
        print(f"Kolumna '{column_name}' nie istnieje w df_info.")

loan_amnt - The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term - The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate - Interest Rate on the loan
installment - The monthly payment owed by the borrower if the loan originates.
grade - LC assigned loan grade
sub_grade - LC assigned loan subgrade
emp_title - The job title supplied by the Borrower when applying for the loan.*
emp_length - Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
home_ownership - The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc - The self-reported annual income provided by the borrower during registration.
verification_status - Indicates if income was verified b

###Analiza brakujących wartości wraz z ich uzupełnieniem/usunięciem przy wzięciu pod uwagę pewnych ustalonych kryteriów

In [141]:
# "pub_rec"
#Wypełnianie zerami wpisów, w których zmienna pubrec nie zawera informacji o
#negatywnych wpisach w rejestrze publicznym
df["pub_rec"].fillna(0, inplace=True)

In [142]:
# "pub_rec_bankruptcies"
#Wypełnienie zerami kolumny nt. bankructw wpisanych do rejestru publicznego
df["pub_rec_bankruptcies"].fillna(0, inplace=True)

In [143]:
# "emp_length"
#Uzupełnianie pustych wartości modą
df['emp_length'].fillna(df['emp_length'].mode().iloc[0], inplace=True)

In [144]:
# "emp_title"
#Z racji dużej ilości pustych wpisów tej cechy, oraz fakt, że jest to zmienna
#typu 'object' oraz ma dużo unikalnych wartości to postanawiam tą zmienną usunąć
display_info(df, ['emp_title'], df_info)
df.drop('emp_title', inplace=True, axis=1)

Column Name: emp_title
Description: The job title supplied by the Borrower when applying for the loan.*

Statistics:
count       39909
unique      30658
top       US Army
freq          139
Name: emp_title, dtype: object

Type:
object

Mode:
0    US Army
Name: emp_title, dtype: object

NaN count: 2627



In [145]:
# "open_acc"
display_info(df, ['open_acc'], df_info)
#Uzupełnianie pustych wartości modą
df["open_acc"].fillna(df["open_acc"].mode().iloc[0], inplace=True)

Column Name: open_acc
Description: The number of open credit lines in the borrower's credit file.

Statistics:
count    42506.000000
mean         9.343951
std          4.496274
min          1.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         47.000000
Name: open_acc, dtype: float64

Type:
float64

Mode:
0    7.0
Name: open_acc, dtype: float64

NaN count: 30



In [146]:
# "purpose"
display_info(df, ['purpose'], df_info)
#Uzupełnianie pustych wartości modą
df["purpose"].fillna(df["purpose"].mode().iloc[0], inplace=True)

Column Name: purpose
Description: A category provided by the borrower for the loan request. 

Statistics:
count                  42535
unique                    14
top       debt_consolidation
freq                   19776
Name: purpose, dtype: object

Type:
object

Mode:
0    debt_consolidation
Name: purpose, dtype: object

NaN count: 1



In [147]:
# "revol_util"
display_info(df, ['revol_util'], df_info)
#Uzupełnianie pustych wartości modą
df["revol_util"].fillna(df["revol_util"].mode().iloc[0], inplace=True)

Column Name: revol_util
Description: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

Statistics:
count     42445
unique     1119
top           0
freq       1070
Name: revol_util, dtype: object

Type:
object

Mode:
0    0
Name: revol_util, dtype: object

NaN count: 91



In [148]:
# "last_credit_pull_d"
display_info(df, ['last_credit_pull_d'], df_info)
#Uzupełnianie pustych wartości modą
df["last_credit_pull_d"].fillna(df["last_credit_pull_d"].mode().iloc[0], inplace=True)

Column Name: last_credit_pull_d
Description: The most recent month LC pulled credit for this loan

Statistics:
count        42531
unique         133
top       Aug-2018
freq          9248
Name: last_credit_pull_d, dtype: object

Type:
object

Mode:
0    Aug-2018
Name: last_credit_pull_d, dtype: object

NaN count: 5



In [149]:
# "annual_inc"
display_info(df, ['annual_inc'], df_info)
#Uzupełnianie pustych wartości modą
df["annual_inc"].fillna(df["annual_inc"].mode().iloc[0], inplace=True)

Column Name: annual_inc
Description: The self-reported annual income provided by the borrower during registration.

Statistics:
count    4.253100e+04
mean     6.913656e+04
std      6.409635e+04
min      1.896000e+03
25%      4.000000e+04
50%      5.900000e+04
75%      8.250000e+04
max      6.000000e+06
Name: annual_inc, dtype: float64

Type:
float64

Mode:
0    60000.0
Name: annual_inc, dtype: float64

NaN count: 5



In [150]:
# "loan_amnt"
display_info(df, ['loan_amnt'], df_info)
#Uzupełnianie pustych wartości modą
df["loan_amnt"].fillna(df["loan_amnt"].mode().iloc[0], inplace=True)

Column Name: loan_amnt
Description: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

Statistics:
count    42535.000000
mean     11089.722581
std       7410.938391
min        500.000000
25%       5200.000000
50%       9700.000000
75%      15000.000000
max      35000.000000
Name: loan_amnt, dtype: float64

Type:
float64

Mode:
0    10000.0
Name: loan_amnt, dtype: float64

NaN count: 1



In [151]:
# "loan_status"
display_info(df, ['loan_status'], df_info)
#Uzupełnianie pustych wartości modą
df["loan_status"].fillna(df["loan_status"].mode().iloc[0], inplace=True)

Column Name: loan_status
Description: Current status of the loan

Statistics:
count          42535
unique             2
top       Fully Paid
freq           36246
Name: loan_status, dtype: object

Type:
object

Mode:
0    Fully Paid
Name: loan_status, dtype: object

NaN count: 1



In [152]:
# "fico_range_low"
display_info(df, ['fico_range_low'], df_info)
#Uzupełnianie pustych wartości modą
df["fico_range_low"].fillna(df["fico_range_low"].mode().iloc[0], inplace=True)

Column Name: fico_range_low
Description: The lower boundary range the borrower’s FICO at loan origination belongs to.

Statistics:
count    42535.000000
mean       713.052545
std         36.188439
min        610.000000
25%        685.000000
50%        710.000000
75%        740.000000
max        825.000000
Name: fico_range_low, dtype: float64

Type:
float64

Mode:
0    685.0
Name: fico_range_low, dtype: float64

NaN count: 1



In [153]:
# "fico_range_high"
display_info(df, ['fico_range_high'], df_info)
#Uzupełnianie pustych wartości modą
df["fico_range_high"].fillna(df["fico_range_high"].mode().iloc[0], inplace=True)

Column Name: fico_range_high
Description: The upper boundary range the borrower’s FICO at loan origination belongs to.

Statistics:
count    42535.000000
mean       717.052545
std         36.188439
min        614.000000
25%        689.000000
50%        714.000000
75%        744.000000
max        829.000000
Name: fico_range_high, dtype: float64

Type:
float64

Mode:
0    689.0
Name: fico_range_high, dtype: float64

NaN count: 1



####fico_range_low oraz fico_range_high

In [154]:
df[['fico_range_low','fico_range_high']].corr()

,fico_range_low,fico_range_high
fico_range_low,1.0,1.0
fico_range_high,1.0,1.0


In [155]:
#Te wartości sa mocno skorelowane więc zamiast dwóch kolumn tworzę jedną z ich średnich
df['fico_mean'] = (df.fico_range_low + df.fico_range_high) / 2
df.drop(['fico_range_low', 'fico_range_high'], axis=1, inplace=True)

In [156]:
mask = df.isna()
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
display(naan_column)
selected_columns = [col_name for col_name in naan_column.columns if ((naan_column[col_name] > 0) & (naan_column[col_name] < 10)).all()]


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt,last_credit_pull_d,verification_status_joint,acc_now_delinq,total_rev_hi_lim,delinq_amnt,pub_rec_bankruptcies,tax_liens,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,fico_mean
NaN,0,1,1,1,1,1,0,1,0,1,0,13294,0,14,1,1,1,30,30,30,0,0,1,0,30,1,1,1,1,0,42536,30,42536,30,0,106,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,0


In [157]:
display_info(df, selected_columns, df_info)

Column Name: term
Description: The number of payments on the loan. Values are in months and can be either 36 or 60.

Statistics:
count          42535
unique             2
top        36 months
freq           31534
Name: term, dtype: object

Type:
object

Mode:
0     36 months
Name: term, dtype: object

NaN count: 1

Column Name: int_rate
Description: Interest Rate on the loan

Statistics:
count      42535
unique       394
top        10.99
freq         970
Name: int_rate, dtype: object

Type:
object

Mode:
0     10.99
Name: int_rate, dtype: object

NaN count: 1

Column Name: installment
Description: The monthly payment owed by the borrower if the loan originates.

Statistics:
count    42535.000000
mean       322.623063
std        208.927216
min         15.670000
25%        165.520000
50%        277.690000
75%        428.180000
max       1305.190000
Name: installment, dtype: float64

Type:
float64

Mode:
0    311.11
Name: installment, dtype: float64

NaN count: 1

Column Name: grade
Descr

In [158]:
for col_name in selected_columns:
    if df[col_name].dtype == 'object':
        df[col_name].fillna(df[col_name].mode().iloc[0], inplace=True)
    elif df[col_name].dtype == np.float64:
        df[col_name].fillna(df[col_name].mean(), inplace=True)


In [159]:
mask = df["loan_amnt"].isna()
df.loc[mask]['loan_amnt']

Series([], Name: loan_amnt, dtype: float64)

###Usuwanie pustych wierszy

In [160]:
df_naan_rows = pd.DataFrame({'Ilość NaN w wierszach': mask.sum(axis=1)})
df_naan_rows

ValueError: ignored

In [ ]:
selected_rows = df_naan_rows[df_naan_rows['Ilość NaN w wierszach'] > 0.65*mask.shape[1]]
selected_rows

####Uzupełnianie wartości

In [ ]:
#inq_last_6mths, delinq_2yrs, pub_rec :
#Brakujące dane zastępuje 0, zakładam że brak informacji o
#zapytaniach oznacza ich brak.

# df['inq_last_6mths'].fillna(0, inplace=True)
# df['delinq_2yrs'].fillna(0, inplace=True)
# df['pub_rec'].fillna(0, inplace=True)

In [ ]:
#mths_since_last_delinq :
##Brakujące dane zastępuje 999, zakładam że brak informacji o
#zapytaniach oznacza, że okres ten jest bardzo długi.

# df['mths_since_last_delinq'].fillna(999, inplace=True)


In [ ]:
# Wykres histogramu zmiennej 'dti'
plt.figure(figsize=(5, 3))
sns.histplot(df['dti'].dropna(), bins=30, kde=True)
plt.xlabel('Wartość dti')
plt.ylabel('Liczba obserwacji')
plt.title('Histogram zmiennej dti')
plt.show()

In [ ]:
from scipy.stats import shapiro

stat, p = shapiro(df['dti'].dropna())

print(f'Statystyka testowa: {stat}')
print(f'P-wartość: {p}')

if p > 0.05:
    print('Brak dowodów na odrzucenie hipotezy o normalności danych.')
else:
    print('Odrzucenie hipotezy o normalności danych.')

In [ ]:
# Wykres histogramu zmiennej "open_acc"
plt.figure(figsize=(10, 6))
sns.histplot(df["open_acc"].dropna(), bins=30, kde=True)
plt.xlabel('Wartość open_acc')
plt.ylabel('Liczba obserwacji')
plt.title('Histogram zmiennej open_acc')
plt.show()